# **Notebook 03 - Modélisation IA : Prédiction des gains CO₂**

**✨ Objectif du notebook**
Ce notebook vise à prédire l'évolution de la part de marché des VE dans les prochaines années afin d'estimer les gains d'émissions de CO₂ en comparaison avec les véhicules thermiques. Nous utilisons un modèle de séries temporelles : **Prophet**.


In [67]:
# Imports
import pandas as pd
import numpy as np
from prophet import Prophet

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go



## 1. Chargement des données nettoyées depuis l'exploration

In [ ]:
df_clean = pd.read_csv("../data/prepared/df_clean.csv")
df_intensite_fr = pd.read_csv("../data/prepared/df_intensite_fr.csv")
df_part_ve = pd.read_csv("../data/prepared/df_part_annee.csv")
df_points_charge = pd.read_csv("../data/prepared/df_points_yearly.csv")
df_points_part = pd.read_csv("../data/prepared/df_points_part.csv")
df_fusion = pd.read_csv("../data/prepared/df_fusion.csv")

## Préparation des données pour Prophet

In [72]:
# Prophet exige un format avec colonnes "ds" (date) et "y" (valeur à prédire)
df_prophet = df_part_ve.copy()
df_prophet.rename(columns={"annee": "ds", "part_ve": "y"}, inplace=True)

# Conversion en datetime (Prophet accepte uniquement les dates)
df_prophet["ds"] = pd.to_datetime(df_prophet["ds"], format="%Y")


## Prédiction avec Prophet

In [73]:
from prophet import Prophet

# Initialisation du modèle
model = Prophet(yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)

# Entraînement
model.fit(df_prophet)

# Création d’un futur dataframe pour 10 ans supplémentaires
future = model.make_future_dataframe(periods=10, freq='Y')

# Prédiction
forecast = model.predict(future)


04:33:10 - cmdstanpy - INFO - Chain [1] start processing
04:33:10 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\orose\anaconda3\envs\ia_project\lib\site-packages\prophet\forecaster.py:1854: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Ici, Prophet apprend à partir des tendances historiques et extrapole sur 10 années futures.

In [76]:
fig = px.line(forecast, x="ds", y="yhat", title="Prévision de la part de marché des VE (%)")
fig.add_scatter(x=df_prophet["ds"], y=df_prophet["y"], mode='markers+lines', name="Données réelles")
fig.update_layout(xaxis_title="Année", yaxis_title="Part de marché VE (%)")
fig.show()


### Estimation de l’impact environnemental projeté

In [77]:
# Hypothèse d’émission moyenne thermique actuelle
emission_thermique = 230  # gCO₂/km

# Hypothèse d’émission VE actuelle (incluant production) : moyenne fixe
emission_ve_total = 49.33 + 83.6  # gCO₂/km usage + fabrication "intégrée"

# Calcul du gain par km
gain_co2_km = emission_thermique - emission_ve_total  # gCO₂ économisé par km

# Estimation d’impact à partir de la part de marché prévue
forecast["co2_evite_par_km"] = forecast["yhat"] * gain_co2_km * 100  # *100 car part en %

# Affichage des 10 prochaines années
forecast[["ds", "yhat", "co2_evite_par_km"]].tail(10)


,ds,yhat,co2_evite_par_km
14,2023-12-31,0.173382,1683.019496
15,2024-12-31,0.189288,1837.416200
16,2025-12-31,0.205150,1991.391055
17,2026-12-31,0.221012,2145.365910
18,2027-12-31,0.236874,2299.340765
19,2028-12-31,0.252780,2453.737469
20,2029-12-31,0.268642,2607.712324
21,2030-12-31,0.284505,2761.687178
22,2031-12-31,0.300367,2915.662033
23,2032-12-31,0.316273,3070.058737


### ℹ️ Justification des hypothèses utilisées

* **Émission thermique : 230 gCO₂/km**
  Cette valeur correspond à une estimation réaliste des émissions d’un véhicule thermique (essence ou diesel) en usage réel, en cohérence avec les données de l’ADEME et les écarts constatés entre tests en laboratoire et conditions routières.

* **Émission VE : 49.33 + 83.6 = 132.93 gCO₂/km**

  * `49.33 gCO₂/km` provient de notre calcul de moyenne sur les véhicules électriques du fichier ADEME (usage).
  * `83.6 gCO₂/km` correspond à l’empreinte carbone de fabrication d’un VE (châssis + batterie), ramenée à une durée de vie de 150 000 km, soit environ 12,5 t CO₂ réparties :

    $$
    \frac{12\,500\ \text{kg CO₂}}{150\,000\ \text{km}} \approx 83.6\ \text{gCO₂/km}
    $$

Ces valeurs sont cohérentes avec les estimations publiées par l’ADEME, l’ICCT et Transport & Environment.


In [79]:
# Visualisation de la prévision de la réduction de CO₂ grâce à l’essor des VE
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=forecast["ds"],
    y=forecast["co2_evite_par_km"],
    mode='lines+markers',
    name="CO₂ évité par km (g/km)",
    line=dict(color='green')
))

fig.update_layout(
    title="Prévision de la réduction de CO₂ grâce à l’essor des VE",
    xaxis_title="Année",
    yaxis_title="g de CO₂ évités / km (selon part de marché VE)",
    template="plotly_white"
)

fig.show()


Le graphe montre la quantité de CO₂ non émise chaque année grâce à l’adoption croissante des véhicules électriques, exprimée par kilomètre parcouru.
Lecture : On observe une augmentation régulière du CO₂ évité par kilomètre parcouru entre 2023 et 2032, en lien avec l’adoption croissante des véhicules électriques (prévue par le modèle Prophet).

### Projection du CO₂ évité cumulé (en tonnes)

Estimer, année après année, la quantité totale de CO₂ évitée grâce à l’adoption croissante des véhicules électriques (VE), en tenant compte d’un gain moyen par VE.

In [80]:
# 'co2_evite_par_km' est en g/km et on convertit en tonnes
forecast.rename(columns={"ds": "annee"}, inplace=True)
forecast["co2_evite_par_km_tonnes"] = forecast["co2_evite_par_km"] / 1e6

# Hypothèse : distance moyenne parcourue par véhicule par an
km_par_an = 13000

# Projection du CO2 évité par an (en tonnes)
forecast["co2_evite_total_tonnes"] = forecast["co2_evite_par_km_tonnes"] * km_par_an

# Cumul des gains dans le temps
forecast["co2_evite_cumule_tonnes"] = forecast["co2_evite_total_tonnes"].cumsum()



In [82]:
fig = px.line(
    forecast,
    x="annee",
    y="co2_evite_cumule_tonnes",
    title="CO₂ évité cumulatif grâce à l'adoption des VE",
    labels={"annee": "Année", "co2_evite_cumule_tonnes": "Tonnes de CO₂ évitées"},
    markers=True
)

fig.update_traces(line=dict(width=3, color="green"))
fig.update_layout(template="plotly_white")
fig.show()


### Scénarios d’adoption VE (conservateur vs ambitieux)

Comparer plusieurs scénarios de part de marché VE (croissance lente, moyenne, rapide) et observer leur impact sur les émissions évitées.

In [83]:
# Création de 3 scénarios
annees = forecast["annee"]
scenarios = {
    "conservateur": forecast["yhat"] * 0.8,
    "réaliste": forecast["yhat"],  # scénario actuel
    "ambitieux": forecast["yhat"] * 1.2
}

# Supposons que le parc total reste stable à 1 000 000 de nouveaux véhicules/an
parc_annuel = 1_000_000
gain_par_ve = forecast["co2_evite_par_km_tonnes"] * km_par_an

# Compilation des scénarios
df_scenarios = pd.DataFrame({"annee": annees})
for nom, part in scenarios.items():
    nb_ve = part * parc_annuel
    co2_evite = nb_ve * gain_par_ve
    df_scenarios[nom] = co2_evite.cumsum()


In [84]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_scenarios["annee"],
    y=df_scenarios["conservateur"],
    mode="lines+markers",
    name="Scénario conservateur",
    line=dict(color="red")
))

fig.add_trace(go.Scatter(
    x=df_scenarios["annee"],
    y=df_scenarios["réaliste"],
    mode="lines+markers",
    name="Scénario réaliste",
    line=dict(color="orange")
))

fig.add_trace(go.Scatter(
    x=df_scenarios["annee"],
    y=df_scenarios["ambitieux"],
    mode="lines+markers",
    name="Scénario ambitieux",
    line=dict(color="green")
))

fig.update_layout(
    title="Scénarios comparés de CO₂ évité (cumulé)",
    xaxis_title="Année",
    yaxis_title="Tonnes de CO₂ évitées",
    template="plotly_white"
)

fig.show()


### Corrélation entre adoption VE, bornes et CO₂ évité

Observer visuellement si la hausse des parts de marché VE et du nombre de bornes semble synchronisée avec l’augmentation du CO₂ évité

In [103]:
# Assure-toi que df_points_charge contient la colonne 'annee'
df_corr = pd.merge(forecast, df_points_charge[["annee", "total_points"]], on="annee", how="inner")

In [104]:
fig1 = go.Figure()

# CO₂ évité cumul (axe gauche)
fig1.add_trace(go.Scatter(
    x=df_corr["annee"],
    y=df_corr["co2_evite_cumule_tonnes"],
    name="CO₂ évité (tonnes)",
    mode="lines+markers",
    line=dict(color="green"),
    yaxis="y"
))

# Part de marché VE (axe droit)
fig1.add_trace(go.Scatter(
    x=df_corr["annee"],
    y=df_corr["yhat"] * 100,
    name="Part VE (%)",
    mode="lines+markers",
    line=dict(color="orange"),
    yaxis="y2"
))

fig1.update_layout(
    title="Évolution du CO₂ évité et de la part de marché VE",
    xaxis=dict(title="Année"),
    yaxis=dict(
        title=dict(text="CO₂ évité (tonnes)", font=dict(color="green")),
        tickfont=dict(color="green")
    ),
    yaxis2=dict(
        title=dict(text="Part VE (%)", font=dict(color="orange")),
        tickfont=dict(color="orange"),
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.5, y=1.15, orientation="h", xanchor="center"),
    template="plotly_white",
    height=500
)

fig1.show()


In [105]:
fig2 = go.Figure()

#  CO₂ évité (axe gauche)
fig2.add_trace(go.Scatter(
    x=df_corr["annee"],
    y=df_corr["co2_evite_total_tonnes"],
    mode="lines+markers",
    name="CO₂ évité (tonnes)",
    line=dict(color="green"),
    yaxis="y1"
))

#  Points de charge (axe droit)
fig2.add_trace(go.Scatter(
    x=df_corr["annee"],
    y=df_corr["total_points"],
    mode="lines+markers",
    name="Points de charge",
    line=dict(color="blue"),
    yaxis="y2"
))

#  Mise en page du graphe
fig2.update_layout(
    title="CO₂ évité vs. Nombre de points de charge",
    xaxis=dict(title="Année"),
    yaxis=dict(
        title="CO₂ évité (tonnes)",
        tickfont=dict(color="green")
    ),
    yaxis2=dict(
        title="Points de charge",
        tickfont=dict(color="blue"),
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.01, y=1.05, orientation="h"),
    template="plotly_white",
    width=900,
    height=500
)

fig2.show()
